In [1]:
# GramSchmidt
# Author: Zhang Su (Teaching Assistant)
# Using python3, numpy
# 3 June 2020

# Learning Outcome

**The code in this demo largely beyonds the scope of the lecture. But it provides lots of facts that might interest you.** 

**Though this demo is mostly for your interest, the Practice is helpful and important for furthering your understanding.** 

By the end of this material, you should be able to:

+ Describe the difference between the theoretical and practical GS process and the way to alleviate the trouble caused by rounding errors.
+ Witness the comparison of the QR factorization using the classical and modified GS process.

Note: 
1. If you occasionally double clicked a textual cell, the display would change to markdown source code. To reverse, simply click anywhere of that markdown cell,  and then click **Run** in the top manu.
2. Sometimes the notebook may not be responding. That is caused by the failure of jupyter kernel. To repair, try clicking **Kernel** in the top manu, then clicking **Reconnect**. 
3. Section Takeaways summarizes useful tips, e.g., holes of Python to avoid, if any.
4. Section Practice reflect the learning outcomes. You are expected to solve them based on your understanding on the lecture notes alone with the coding skills learned from this demo.

# Table of contents <a name="Table_of_Content"></a>
+ 6.3.2. [Gram_Schmidt](#Gram_Schmidt) 
+ 6.3.3. [QR Factorization by Gram Schmidt](#QR_Factorization)
+ [Classical Gram-Schmidt VS. Modified Gram-Schmidt](#vs) (For your interests)
+ [Takeaways](#Takeaways)
+ [Practice](#Practice)


Let's import the libraries.

In [2]:
import numpy as np
from numpy import linalg as la

### Gram-Schmidt Process<a name="Orthonormal_Sets"></a>
[Return to Table of Content](#Table_of_Content)

**This part of code is for Section 6.3.2.**

![title](img/gram_schmidt.png)
> Figure 1. The Gram-Schmidt process. Note that the process shown in the figure does not include the normalization. Though $Q=\{v_1,v_2,\ldots,v_p\}$ can already form an orthogonal set, it would be better if they are further normalized to be an orthonormal set. Tutorial 6.2 provides plenty of explanations on this.

Let's implement the GS process following the notation of Fig. 1. Given a $m\times n$ matrix $A=\{x_1, x_2, \ldots, x_p\}$, the goal is to obtain a new $m\times n$ matrix $Q=\{v_1,v_2,\ldots,v_p\}$, whose columns are mutually orthonormal.

In [3]:
def gram_schmidt(A):
# The classical Gram Schmidt process.
# Input: an arbitrary mxp matrix.
# Output: the Q factors of the input.   

    n = A.shape[1]
    Q = np.zeros(A.shape)
    
    for i in range(n):
        xi = A[:, i]
        vi = xi
        for j in range(i):
            # Since Q[:,j] is always a unit vector,
            # no need to implement the denominator. 
            vi = vi - np.dot(xi, Q[:,j])*Q[:,j] 

        Q[:, i] = vi/la.norm(vi)
    
    return Q

You may wonder why the denominator is missing in Line 10. The reason is that in the first iteration, i.e., when `i=0`, the nested iteration in Line 9 will not trigger. Therefore, the only thing achieved in the first iteration is the normalization of $x_1$, which is saved as $v_1$. In the second (and further iterations), `Q[:, j]` in Line 10 equals the normalized $v_1$. Since $v_1\cdot v_1=1$, the denominator can be omitted. 

In [4]:
A = np.array([[3,8],
            [0,5],
            [-1,-6]], dtype=float)

Q = gram_schmidt(A)

print("The Gram-Schmidt Process yields \n", Q)
print("Is Q orthonormal?", np.allclose(Q.T.dot(Q), np.eye(Q.shape[1])))

The Gram-Schmidt Process yields 
 [[ 0.9486833  -0.16903085]
 [ 0.          0.84515425]
 [-0.31622777 -0.50709255]]
Is Q orthonormal? True


### QR Factorization by Gram Schmidt<a name="QR_Factorization"></a>
[Return to Table of Content](#Table_of_Content)


Now we come to the QR factorization (using Gram-Schmidt process)! Actually, there are many ways to implement the Gram-Schmidt process as shown in Fig. 1. Here are two *very different* implementations:

![title](img/compare_gram.png)
>Figure 2. The comparison of two typical implementations.

They are arithmetically equivalent, i.e., they do the same task as shown in Fig. 1. In exact arithmetic the two implementations are identical, while in the presence of rounding errors they are dramatically different.

In classical Gram-Schmidt, we compute the length of the orthogonal projections of $w=a_k$ onto $q_1, q_2, \ldots, q_{k-1}$, and then subtract those projections (with the rounding errors) from $w$. But because of rounding errors, $q_1, q_2, \ldots, q_{k-1}$ are not truly orthogonal. It accumulates errors.

In modified Gram-Schmidt, we compute the length of the projection of $w=a_k$ onto $q_1$ and subtract that projection (and the rounding errors) from $w$. Next we compute the length of the projection of the **computed** $w$ onto $q_2$ and subtract that projection (and the rounding errors) from $w$, and so on, but always orthogonalizing against the **computed version of $w$**.

The difference only emerges when $k>=3$.

Let us implement them first and run some comparisons. Note that in Fig. 1, the output is the Q factor of QR factorization. In our implementations we would also output the R factor following Fig. 2.

Be ware of the pythonic [**memory issue**](https://ipython-books.github.io/45-understanding-the-internals-of-numpy-to-avoid-unnecessary-array-copying/), the operations of any `Q[i, j] = z` affect the memory block. An example is provided in **Section Takeaways**.

Also, the `qr_gram_schmidt()` implemented below is an extension of `gram_schmidt()` in the first part. Compared to `gram_schmidt()`, `qr_gram_schmidt()` is written in a more economical manner, with the factor R outputted. 

**Challenge: can you revise `gram_schmidt()` following the idea of modified Gram-Schmidt discribed above?** You are expected to revise the routine `gram_schmidt()` provided in the first part of this demo. No need to output the R factor. (Don't worry, this is only for your interest :)

In [5]:
def qr_gram_schmidt(A):
    # The QR factorization using the classical Gram Schmidt process.
    # Input: an arbitrary mxp matrix.
    # Output: the Q and R factors of the input.

    # Obtain the column number.
    p = A.shape[1]

    # Initiate the Q and R factors.
    Q = A.copy() # Prevent A from being changed.
    R = np.zeros((p, p))

    # Iterate for each column of the input.
    for k in range(p):
        w = A[:, k]
        
        ##### Different part. #####
        for i in range(k):
            R[i, k] = Q[:, i].dot(w)

        for i in range(k):
            w = w - R[i, k] * Q[:, i]
        ###########################
        
        R[k, k] = la.norm(w)
        Q[:, k] = w / R[k, k]

    return Q, R

In [6]:
def qr_mod_gram_schmidt(A):
# The QR factorization using the modified Gram Schmidt process.
# Input: an arbitrary mxp matrix.
# Output: the Q and R factors of the input.

    # Obtain the column number.
    p = A.shape[1]
    
    # Initiate the Q and R factors.
    Q = A.copy() # Prevent A from being changed.
    R = np.zeros((p,p))
    
    # Iterate for each column of the input.
    for k in range(p):
        w = A[:, k]
        
        ##### Different part. #####
        for i in range(k):
            R[i, k] = Q[:, i].dot(w)
            w = w - R[i, k] * Q[:, i]
        ###########################
        
        R[k, k] = la.norm(w)
        Q[:, k] = w / R[k, k]
            
    return Q, R

### Classical Gram-Schmidt VS. Modified Gram-Schmidt<a name="vs"></a>
[Return to Table of Content](#Table_of_Content)

A simple experiment is carried out to compare the performance of these two algorithms.

#### Critera

We employ the reconstruction error and orthogonal error to measure the performance of the QR factorization implementations. The motivations are explained as follows.

Given a square matrix $A$, we carry out QR factorization and obtain $A=QR$. There are two properties we perticularly expect. 

1. The two factors $Q$ and $R$ can reconstruct $A$ precisely.
2. The $Q$ factor holds the orthonormality.

For property No. 1, we can examine the matrix $A-QR$. Ideally, $A-QR$ should yield a matrix with all zeros. For property No. 2, we can examine the matrix $I-Q^TQ$. Ideally, $I-Q^TQ$ should also be a zero matrix.

Yet, reality is always so crucial, no? We will soon see that the matrices $A-QR$ and $I-Q^TQ$ are not valued as expected. To measure to what extent can the actual $Q$ and $R$ hold the properities, we employ **infinity norm**. So what is it? Let's see two examples.

Given a vector $u=(1,2,-3,4,-5)$, its infinity norm $\|u\|_\infty=5$. Here, $5$ is the absolute value of the fifth entry. Of all the entries, $-5$ has the largest absolute value. In sum, the infinity norm for a vector returns the absolute of the entry with the largest absolute value.

Given a matrix $B=\begin{pmatrix}
1& -7\\
-2 & -3
\end{pmatrix}$, its infinity norm $\|A\|_\infty=8$. Here, $8$ is the sum of the absolute values of the first row. Of all the rows, the first row has the largest absolute value, i.e., $|1|+|-7|=1+7=8>|-2|+|-3|=5$. In sum, the infinity norm for a matrix returns the sum of absolute values of the row with the largest sum of absolute values over its entries.


In this demo, we employ the infinity norm of matrix, because we are interested to see the most deviant row in matrix $A-QR$ and $I-Q^TQ$. Since the entries of these two matrix might be mostly approaching 0, using our farmiliar 2-norm may not be able to reveal the difference.

Note, I deliberately avoid of formal mathematical notations. You may refer to [this link](https://www.netlib.org/lapack/lug/node75.html) for more solid definitions or examples of all norms.

In [7]:
def compare(Q, R, A):
# Input: the Q, and R factors of A, and the A matrix.
# Output: reconstruction_error is the infinity norm of matrix (QR - A).
#         orthogonal_error is the infinity norm of matrix (QtQ-I)
# Given a matrix, the infinity norm returns the sum of 
# absolute values of the row with the largest sum of absolute 
# values over its entries.

    reconstruction_error = la.norm(Q.dot(R) - A, ord=np.inf) / la.norm(A, ord=np.inf)
    orthogonal_error = la.norm(Q.T.dot(Q)-np.eye(R.shape[0]), ord=np.inf)
    return reconstruction_error, orthogonal_error

#### On a matrix with two columns

First let's try the `3x2` matrix from Q7 of Tutorial 6.3. According to the pseudo-code of Fig. 2, there should be no differece (Difference emerges only when column number is greater than 2). Note that you have to **specify the data type** as `dtype=float` for the matrix. Otherwise, the matrix `A` will be considered as an integer data (since we manyally assigned integers), which may produce wrong output from dividing. We shall obtain an orthonormal version of Q compared to the solution of this tutorial question.

In [8]:
A = np.array([[3,8],
            [0,5],
            [-1,-6]], dtype=float)

Q, R = qr_gram_schmidt(A)

reconstruction_error, orthogonal_error = compare(Q, R, A)
print('The should-be-orthonormal matrix Q =\n', Q.round(4))
print('The reconstruction error =\n', reconstruction_error)
print('The orthogonal error =\n', orthogonal_error)

Q, R = qr_mod_gram_schmidt(A)

reconstruction_error, orthogonal_error = compare(Q, R, A)
print('The should-be-orthonormal matrix Q =\n', Q.round(4))
print('The reconstruction error =\n', reconstruction_error)
print('The orthogonal error =\n', orthogonal_error)

The should-be-orthonormal matrix Q =
 [[ 0.9487 -0.169 ]
 [ 0.      0.8452]
 [-0.3162 -0.5071]]
The reconstruction error =
 0.0
The orthogonal error =
 1.3877787807814457e-16
The should-be-orthonormal matrix Q =
 [[ 0.9487 -0.169 ]
 [ 0.      0.8452]
 [-0.3162 -0.5071]]
The reconstruction error =
 0.0
The orthogonal error =
 1.3877787807814457e-16


#### On a matrix with three columns

Next let's try the 3x3 matrix from one of our tutorial questions. Since `A` is very well specified, both two methods can obtain perfect results.

In [9]:
A = np.array([[1,2, 4],
            [0,0,5],
            [0,3,6]], dtype=float)

Q, R = qr_gram_schmidt(A)

reconstruction_error, orthogonal_error = compare(Q, R, A)
print('The should-be-orthonormal matrix Q =\n', Q.round(4))
print('The reconstruction error =\n', reconstruction_error)
print('The orthogonal error =\n', orthogonal_error)

Q, R = qr_mod_gram_schmidt(A)

reconstruction_error, orthogonal_error = compare(Q, R, A)
print('The should-be-orthonormal matrix Q =\n', Q.round(4))
print('The reconstruction error =\n', reconstruction_error)
print('The orthogonal error =\n', orthogonal_error)

The should-be-orthonormal matrix Q =
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
The reconstruction error =
 0.0
The orthogonal error =
 0.0
The should-be-orthonormal matrix Q =
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
The reconstruction error =
 0.0
The orthogonal error =
 0.0


#### On a 50-by-50 Helbert matrix 

Let's try something "harder", a $50\times 50$ [Hilbert matrix](https://en.wikipedia.org/wiki/Hilbert_matrix)! We can see that since the matrix is [not well conditioned](https://en.wikipedia.org/wiki/Condition_number), the loss on orthogonality are both obvious, where the result from the modified one is relatively good. (We can also have a sense of how terrible the accumulated rounding errors can be!)

In [10]:
from scipy.linalg import hilbert as hilb

n = 50
A = hilb(n)

Q, R = qr_gram_schmidt(A)
reconstruction_error, orthogonal_error = compare(Q, R, A)
print('The reconstruction error =\n', reconstruction_error)
print('The orthogonal error =\n', orthogonal_error)

Q, R = qr_mod_gram_schmidt(A)
reconstruction_error, orthogonal_error = compare(Q, R, A)
print('The reconstruction error =\n', reconstruction_error)
print('The orthogonal error =\n', orthogonal_error)

The reconstruction error =
 1.010172945062073e-16
The orthogonal error =
 42.85021637056607
The reconstruction error =
 9.63905481929459e-17
The orthogonal error =
 4.666330075235493


#### On a very bad conditioned matrix

Finally, lets try a very special full-rank matrix:

$$
A=\left[\begin{array}{rrr}
1 & 1 & 1 \\
0.00000001 & 0.00000001 & 0 \\
0.00000001 & 0 & 0.00000001
\end{array}\right]
$$

We will see that the *should-be-orthonormal* matrix $Q$ produced by the classical Gram-Schmidt is not orthonormal at all! 

As shown in the results, **the second and third columns of Matrix $Q$** from the classicial Gram-Schmidt have a $45^\circ$ angle, which is a terrible failure!

In [11]:
eps = 1e-8

A = np.array([
    [1,  1,  1],
    [eps,eps,0],
    [eps,0,  eps]
    ])

Q, R = qr_gram_schmidt(A)
reconstruction_error, orthogonal_error = compare(Q, R, A)

print('The reconstruction error =\n', reconstruction_error)
print('The orthogonal error =\n', orthogonal_error)
print('The should-be-orthonormal matrix Q =\n', Q.round(4))
print('The second and third columns above have an angle of 45 degree.\n')

Q, R = qr_mod_gram_schmidt(A)
reconstruction_error, orthogonal_error = compare(Q, R, A)

print('The reconstruction error =\n', reconstruction_error)
print('The orthogonal error =\n', orthogonal_error)
print('The should-be-orthonormal matrix Q =\n', Q.round(4))

The reconstruction error =
 0.0
The orthogonal error =
 0.7071067953286833
The should-be-orthonormal matrix Q =
 [[ 1.      0.      0.    ]
 [ 0.      0.     -0.7071]
 [ 0.     -1.     -0.7071]]
The second and third columns above have an angle of 45 degree.

The reconstruction error =
 0.0
The orthogonal error =
 2e-08
The should-be-orthonormal matrix Q =
 [[ 1.  0.  0.]
 [ 0.  0. -1.]
 [ 0. -1.  0.]]


Note that there exists other implementations for the Gram_Schmidt, e.g., the [householder method](https://atozmath.com/example/MatrixEv.aspx?he=e&q=qrdecomphh). You may dig more if you wish.

### Takeaways<a name="Takeaways"></a>
[Return to Table of Content](#Table_of_Content)

1. The classical and modified Gram-Schmidt are arithmetically equivalent, they are different when we are running them on the computer due to rounding errors.
2. Pay attention to the difference of how Matlab and Python [handle the memory](https://ipython-books.github.io/45-understanding-the-internals-of-numpy-to-avoid-unnecessary-array-copying/), as shown in the next cell. That's why the `A.copy()` is used in the implementation of GS process (Line 10 of `gram_schmidt()` and `mod_gram_schmidt()`). The `.copy()` method prevent the alternation of the original `A`, by which `A` will not be changed after calling the functions.

In [12]:
def memory_handle_test(B1, B2):    
    B1[0,0] = 0  # Assign 0 to the first entry of array B1
    B2 = 0 # Assign 0 to the scalar B2
    C = 2
    return C

##############################################


A1 = np.array([[1,1]])
A2 = 1

print('The original A1 = ', A1)
print('The original A2 = ', A2)

B = memory_handle_test(A1, A2)

print('The actual A1 after calling the function = ', A1)
print('The actual A2 after calling the function = ', A2)

The original A1 =  [[1 1]]
The original A2 =  1
The actual A1 after calling the function =  [[0 1]]
The actual A2 after calling the function =  1


In MATLAB, `A1` and `A2` will not be changed after calling the function. However, in Python, `A1` is changed, because `B1[0,0] = 0` changed the value of the memory block. This may confuse you if you are new to Python from MATLAB, like me :)

### Practice<a name="Practice"></a>
[Return to Table of Content](#Table_of_Content)

**This part can get you ready for the lab!**

Given $A=\begin{pmatrix}
1 & 2 & 4 \\
0 & 0 & 5 \\
0 & 3 & 6 
\end{pmatrix}$ and

Given $B=\begin{pmatrix}
1 & 2 & 4 \\
1 & 2 & 5 \\
1 & 2 & 6 
\end{pmatrix}$.

1. Manually find the rank of $A$ and $B$. using pens with row reduction. Compare your result against `la.matrix_rank()` from `Numpy`.
2. Apply the `qr_gram_schmidt()` on $A$. Print the $Q$ and $R$ factors.
3. Verify that if $Q$ is an orthogonal matrix as we did in Section 6.2.4 of the last demo.
3. Report the rank of the augmented matrix $[A\,\,Q]$ using `Numpy`. What it implies?
4. Apply the `qr_gram_schmidt()` on $B$. Print the $Q$ and $R$ factors.
5. Verify that if $Q$ is an orthogonal matrix as we did in Section 6.2.4 of the last demo.
6. Why it failed on this $B$?
7. Describe how to *adjust* B to achieve a better result? (Column swapping is allowed only)
8. `numpy.linalg.qr()` is the routine from Numpy for QR factorization. It utilizes other algorithms for orthogonalization so that it can overcome the issue caused by linear dependency. Apply it on the $B$, print the $Q$ and $R$ factors, and verify the orthonormality of $Q$.
9. Report the rank of the augmented matrix $[B\,\,Q]$ using `Numpy` again. What it implies compared to that from question 4?
10. Based on whether the given matrix is linearly independent on columns, conclude the functionality of the QR factorization.